In [27]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
#
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

import tools
import importlib
importlib.reload(tools)
#
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.metrics import r2_score, mean_squared_error

### Download data from AWS S3

In [2]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.download_data_from_s3(s3_bucket, local_path)

### CPU Count

In [3]:
cpu_count()

4

### Begin

In [4]:
# dataset = pd.read_csv('../data/all_stock_close.csv')
dataset = pd.read_csv('../data/us_hk/us_hk_clean_10yr_stock_close.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', drop=True, inplace=True)
dataset.head(3)

,0001.HK,0002.HK,0003.HK,0004.HK,0005.HK,0006.HK,0007.HK,0008.HK,0010.HK,0011.HK,...,VFC,VRNT,WDC,WFC,WMT,^DJI,^GSPC,^IXIC,^TNX,^VIX
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-28,56.334431,41.966953,5.800245,7.799201,49.283875,26.950289,0.71,1.890068,40.080761,82.240196,...,16.199192,31.690001,26.569702,23.259796,41.967972,11575.540039,1258.510010,2662.879883,3.481,17.520000
2010-12-29,57.905922,42.365368,5.895123,8.097270,49.750565,27.142986,0.72,1.935749,40.827366,83.661453,...,16.249552,31.540001,26.420523,23.125603,42.233482,11585.379883,1259.780029,2666.929932,3.341,17.280001
2010-12-30,57.620216,42.199368,5.888801,8.229015,49.812798,27.170521,0.76,1.952879,41.063133,83.726082,...,16.201061,31.500000,26.420523,22.976505,42.225681,11569.709961,1257.880005,2662.979980,3.369,17.520000


In [5]:
train_start_txt = '2011-01-01'
train_end_txt = '2011-12-31'
dataset = dataset[train_start_txt:train_end_txt]

In [6]:
prevday_returns = np.log(dataset.shift(1)/dataset.shift(2))
today_returns = np.log(dataset/dataset.shift(1))
#
stocks_prev = prevday_returns.columns.to_list()
stocks_today_draft = today_returns.columns.to_list()
stocks_today = [s for s in stocks_today_draft if '.HK' in s]
#
new_col = ['{}.prev'.format(c) for c in stocks_prev]
prevday_returns.columns = new_col
#
comb_df = pd.concat([prevday_returns, today_returns[stocks_today]], axis=1)
comb_df.shape

(245, 1525)

### Fina the NaN in comb_df

In [7]:
comb_df.dropna(axis=0, how='any', inplace=True)

In [8]:
comb_df.tail(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3900.HK,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK
Date,,,,,,,,,,,,,,,,,,,,,
2011-12-28,0.012451,0.002274,0.007808,0.011281,0.014316,0.003512,-0.029853,0.054347,0.042260,0.007547,...,-0.048374,0.005141,-0.002238,-0.017483,-0.020762,-0.057175,0.000000,-0.013129,-0.007435,0.000000
2011-12-29,0.004829,-0.007599,-0.001112,-0.025573,-0.010084,0.006987,-0.015268,-0.003839,-0.005764,-0.004845,...,-0.014685,0.005115,-0.009009,-0.008854,0.000000,-0.060644,-0.020495,-0.013304,-0.030305,0.000000
2011-12-30,-0.009142,-0.000763,-0.006697,-0.004321,-0.013606,-0.008741,0.000000,0.041437,0.009206,-0.003243,...,-0.002963,0.000000,0.011247,-0.002545,0.000000,0.459543,-0.017911,-0.018019,0.015268,0.013247


In [9]:
comb_df.head(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3900.HK,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-05,0.012648,-0.005488,0.002165,0.001615,0.003118,0.001009,-0.027029,0.008607,0.000955,-0.001539,...,0.006369,0.000000,0.002567,0.002462,-0.007203,-0.028982,0.010612,0.040191,0.000000,-0.008772
2011-01-06,0.010938,0.004706,-0.001083,-0.011347,0.023994,0.001009,-0.013793,-0.014388,0.004764,0.017564,...,0.015748,0.056834,-0.033907,0.000000,0.002407,0.000000,-0.017302,-0.021440,-0.006473,0.008268
2011-01-07,0.021523,0.004684,-0.014168,0.004068,0.005456,-0.002018,0.013793,0.014388,0.006632,-0.000757,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063791


In [10]:
prevday_returns = comb_df[new_col]
today_returns = comb_df[stocks_today]

In [11]:
print('{} {}'.format(len(new_col), len(stocks_today)))

812 713


### Regression

In [22]:
def get_linear_regression(pair, prevday_returns, today_returns):
    catstocks = pair['prev']
    column = pair['today']
    X = prevday_returns[catstocks].copy()
    X = sm.add_constant(X)
    y = today_returns[column]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstocks,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared,
        'model' : model,
        'y_train' : y,
        'y_train_pred' : model.predict(X)
    }
    # print(data)
    return data

### Generate pair list

In [13]:
def generate_pair_list(new_col, today_returns, best_prev_sofar_df):
    catstocks = new_col
    pair_list = []
    for column in today_returns.columns:
        if best_prev_sofar_df is not None:
            best_prev_str = best_prev_sofar_df.loc[best_prev_sofar_df['Today']==column, 'Prev'].values[0]
            best_prev_list = best_prev_str.replace('\'','').strip('][').split(', ') # convert list representation to list
        else:
            best_prev_list = list()
        for catstock in catstocks:
            if (column not in catstock) and (catstock not in best_prev_list):
                data = {
                    'prev' : best_prev_list + [catstock],
                    'today' : column
                }
                pair_list.append(data)
    return pair_list
#

In [39]:
def feature_score(train_mse, test_mse, train_r2, test_r2):
    score = train_r2 / (train_r2-test_r2+0.1)
    return score

In [14]:
pair_list = generate_pair_list(new_col, today_returns, None)
pair_list[:5]

[{'prev': ['0002.HK.prev'], 'today': '0001.HK'},
 {'prev': ['0003.HK.prev'], 'today': '0001.HK'},
 {'prev': ['0004.HK.prev'], 'today': '0001.HK'},
 {'prev': ['0005.HK.prev'], 'today': '0001.HK'},
 {'prev': ['0006.HK.prev'], 'today': '0001.HK'}]

### Experiment - Train Test split wrapper

In [31]:
train_start = datetime.strptime(train_start_txt, '%Y-%m-%d')
train_end = datetime.strptime(train_end_txt, '%Y-%m-%d')
train_end = train_end - relativedelta(months=3)
test_start = train_end + relativedelta(days=1)
test_end = train_end + relativedelta(months=3)
train_end = train_end + relativedelta(months=3)
print('{} {} {} {}'.format(train_start, train_end, test_start, test_end))

2011-01-01 00:00:00 2011-12-30 00:00:00 2011-10-01 00:00:00 2011-12-30 00:00:00


	Prev	Today	Params	TValues	RSquared
348463	['MFC.prev']	0945.HK	0.690853	18.223915	0.579488

In [35]:
pair = [x for x in pair_list if (x['prev']==['MFC.prev']) and (x['today']=='0945.HK')][0]
pair

{'prev': ['MFC.prev'], 'today': '0945.HK'}

In [36]:
train_data = get_linear_regression(pair, prevday_returns[train_start:train_end],\
                                   today_returns[train_start:train_end])
y_train = train_data['y_train']
y_train_pred = train_data['y_train_pred']
#
X_test = prevday_returns.loc[test_start:test_end, pair['prev']].copy()
X_test = sm.add_constant(X_test)
y_test = today_returns.loc[test_start:test_end, pair['today']]
y_test_pred = train_data['model'].predict(X_test)
#
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [38]:
print('{} {}'.format(train_mse, test_mse))
print('{} {}'.format(train_r2, test_r2))
print('{}'.format(train_data['RSquared']))

0.0002616403058517055 0.0004265721973687229
0.5794881374385089 0.5147848858482975
0.5794881374385088


### Train Test split wrapper

In [44]:
def linear_regression_score(pair, prevday_returns, today_returns, train_start, train_end, test_start, test_end):
    train_data = get_linear_regression(pair, prevday_returns[train_start:train_end],\
                                       today_returns[train_start:train_end])
    y_train = train_data['y_train']
    y_train_pred = train_data['y_train_pred']
    #
    X_test = prevday_returns.loc[test_start:test_end, pair['prev']].copy()
    X_test = sm.add_constant(X_test)
    y_test = today_returns.loc[test_start:test_end, pair['today']]
    y_test_pred = train_data['model'].predict(X_test)
    #
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    #
    data = {
        'Prev' : train_data['Prev'],
        'Today' : train_data['Today'],
        'Params' : train_data['Params'],
        'TValues' : train_data['TValues'],
        'train_mse' : train_mse,
        'test_mse' : test_mse,
        'train_r2' : train_r2,
        'test_r2' : test_r2,
        'score' : feature_score(train_mse, test_mse, train_r2, test_r2)
    }
    print(data)
    return data

### NON-Parallel processing

In [45]:
%%time
results = [linear_regression_score(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list]
result_df = pd.DataFrame(results)
result_df

/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7

{'Prev': ['0002.HK.prev'], 'Today': '0001.HK', 'Params': -0.039702914115834986, 'TValues': 0.3045674049706116, 'train_mse': 0.00038316051042979604, 'test_mse': 0.0006632043009837319, 'train_r2': 0.00038475358475609234, 'test_r2': -0.001233532129157, 'score': 0.0037862632896533276}
{'Prev': ['0003.HK.prev'], 'Today': '0001.HK', 'Params': -0.08394397774228504, 'TValues': 0.758921353304722, 'train_mse': 0.00038239411265328133, 'test_mse': 0.0006601505687308248, 'train_r2': 0.002384184322969607, 'test_r2': 0.0033766598210757115, 'score': 0.024080839663090457}
{'Prev': ['0004.HK.prev'], 'Today': '0001.HK', 'Params': 0.02287331412097389, 'TValues': 0.43956658491701944, 'train_mse': 0.0003830009232784803, 'test_mse': 0.0006716231397856344, 'train_r2': 0.0008010954186478125, 'test_r2': -0.01394337688974101, 'score': 0.006981560004866966}
{'Prev': ['0005.HK.prev'], 'Today': '0001.HK', 'Params': -0.02196309887019115, 'TValues': 0.3261154726918416, 'train_mse': 0.00038313891352744774, 'test_mse':

/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7

{'Prev': ['0025.HK.prev'], 'Today': '0001.HK', 'Params': 0.09573335912462146, 'TValues': 0.9449365279070773, 'train_mse': 0.0003818930756413964, 'test_mse': 0.0006619599173988123, 'train_r2': 0.003691323817947012, 'test_r2': 0.0006451025090751239, 'score': 0.035822020167848724}
{'Prev': ['0026.HK.prev'], 'Today': '0001.HK', 'Params': -0.18756765089943203, 'TValues': 1.588762656196008, 'train_mse': 0.00037933494118697633, 'test_mse': 0.0006512477831453603, 'train_r2': 0.010365159282228142, 'test_r2': 0.01681711466176561, 'score': 0.11080038416923667}
{'Prev': ['0028.HK.prev'], 'Today': '0001.HK', 'Params': 0.0647147112908487, 'TValues': 1.3897638053221613, 'train_mse': 0.00038026047256871576, 'test_mse': 0.0006636303302480655, 'train_r2': 0.007950569952067998, 'test_r2': -0.0018767046545558053, 'score': 0.07239158014751002}
{'Prev': ['0031.HK.prev'], 'Today': '0001.HK', 'Params': 0.06778047049307723, 'TValues': 1.4337950043644512, 'train_mse': 0.0003800659667484297, 'test_mse': 0.000656

/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7

...

/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7

{'Prev': ['0613.HK.prev'], 'Today': '0001.HK', 'Params': 0.03003217923910108, 'TValues': 0.5962090011545125, 'train_mse': 0.0003827434585580994, 'test_mse': 0.0006666696669108729, 'train_r2': 0.0014727869236468294, 'test_r2': -0.006465163712675137, 'score': 0.013644755296578931}
{'Prev': ['0617.HK.prev'], 'Today': '0001.HK', 'Params': -0.04455905448427986, 'TValues': 0.6226528320821895, 'train_mse': 0.00038269235260450924, 'test_mse': 0.0006616027451346342, 'train_r2': 0.0016061156174989044, 'test_r2': 0.0011843222443983503, 'score': 0.015993695825881615}
{'Prev': ['0618.HK.prev'], 'Today': '0001.HK', 'Params': 0.020882743817670853, 'TValues': 0.6041938337458008, 'train_mse': 0.0003827282591659406, 'test_mse': 0.0006734202243673515, 'train_r2': 0.0015124401335279147, 'test_r2': -0.016656419221671825, 'score': 0.012798973788701155}
{'Prev': ['0620.HK.prev'], 'Today': '0001.HK', 'Params': 0.020570935556458975, 'TValues': 0.5782158365470839, 'train_mse': 0.00038277697218620224, 'test_mse'

/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2542: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/opt/miniconda3/envs/conda_env/lib/python3.7

{'Prev': ['0670.HK.prev'], 'Today': '0001.HK', 'Params': 0.0404369063561725, 'TValues': 1.0535853071651167, 'train_mse': 0.00038155057394288327, 'test_mse': 0.0006595865783501886, 'train_r2': 0.004584865585528397, 'test_r2': 0.004228110995402545, 'score': 0.045685670130065206}
{'Prev': ['0672.HK.prev'], 'Today': '0001.HK', 'Params': 0.10546443178594261, 'TValues': 1.8718972213523002, 'train_mse': 0.00037781478332348884, 'test_mse': 0.0006684129094598988, 'train_r2': 0.014331050693103475, 'test_r2': -0.00909692121504646, 'score': 0.11610861356263763}
{'Prev': ['0677.HK.prev'], 'Today': '0001.HK', 'Params': -0.051209068466427525, 'TValues': 0.7831258368684899, 'train_mse': 0.0003823350403546752, 'test_mse': 0.0006625641360162349, 'train_r2': 0.002538296160489728, 'test_r2': -0.00026708087031934724, 'score': 0.02469030544704916}


ValueError: cannot infer type for <class 'NoneType'>

### Parrallel processing

In [ ]:
%%time
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(linear_regression_score)(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list)
results = executor(tasks)
result_df = pd.DataFrame(results)
result_df

In [ ]:
result_df.to_csv('../data/us_hk/us_hk_traintest_1indp_regression_pair.csv')

### Read Regression Result

In [ ]:
result2_df = pd.read_csv('../data/us_hk/us_hk_traintest_1indp_regression_pair.csv', index_col=0)

In [ ]:
sorted2_df = result2_df.sort_values(by='RSquared', ascending=False)
sorted2_df.head(10)

### Upload to AWS S3

In [ ]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.upload_data_to_s3(s3_bucket, local_path)